In [1]:
# import des librairies dont nous aurons besoin
import pandas as pd
import numpy as np
import re

# chargement et affichage des données
data = pd.read_csv('personnes.csv')
print(data)

   prenom                                   email date_naissance  \
0   Leila                       leila@example.com     23/01/1990   
1  Samuel                  samuel_329@example.com     20/09/2001   
2   Radia                choupipoune@supermail.eu  12 sept. 1984   
3    Marc  marco23@example.com, mc23@supermail.eu     10/02/1978   
4    Heri                 helloworld@supermail.eu     05/03/2008   
5   Hanna                  hanna2019@supermail.eu     01/01/1970   
6  samuël                  samuel_329@example.com            NaN   

            pays taille  
0         France  1.49m  
1            NaN  1.67m  
2  Côte d'ivoire  153cm  
3         France  1.65m  
4     Madagascar  1.34m  
5             24  3.45m  
6          Bénin  1.45m  


In [2]:
# EXEMPLE RAPIDE METHODE POUR REMPLACER DES VALEURS DANS UN DATAFRAME
#Fonction qui renvoit une minuscule
def lower_case(value): 
    print('Voici la valeur que je traite:', value)
    return value.lower()

datatest = pd.DataFrame([['A',1],
                     ['B',2],
                     ['C',3]], columns = ['lettre','position'])

# la methode .apply utilise une fonction sur l'ensemble d'une serie !
nouvelle_colonne = datatest['lettre'].apply(lower_case)
nouvelle_colonne = nouvelle_colonne.values
print(nouvelle_colonne)
datatest['lettre'] = nouvelle_colonne
print(datatest)

Voici la valeur que je traite: A
Voici la valeur que je traite: B
Voici la valeur que je traite: C
['a' 'b' 'c']
  lettre  position
0      a         1
1      b         2
2      c         3


In [3]:
# TRAITEMENT DE LA COLONNE COUNTRY : erreur lexicale
# verifier une valeur // dictionnaire
VALID_COUNTRIES = ['France', 'Côte d\'ivoire', 'Madagascar', 'Bénin', 'Allemagne'
                  , 'USA']
#fonction de Correction                  
def check_country(country):
    if country not in VALID_COUNTRIES:
        print(' - "{}" n\'est pas un pays valide, nous le supprimons.' \
            .format(country)) #permet de vérifier les corrections
        return np.NaN
    return country

# TRAITEMENT DE LA COLONNE EMAIL : Erreur de supposition (2 valeurs au lieu d'une)
# ne garder que la premiere valeur
def first(string):
    parts = string.split(',')
    first_part = parts[0]
    if len(parts) >= 2:
        print(' - Il y a plusieurs parties dans "{}", ne gardons que {}.'\
            .format(parts,first_part))  
    return first_part

# TRAITEMENT DES TAILLES : ERREUR d'IRREGULARITE (Bonne valeurs, mauvaise unité)
# + conversion en colonne chiffre
def convert_height(height):
    found = re.search('\d\.\d{2}m', height) # EXPRESSION REGULIERE, regarder regles associées
    if found is None:
        print('{} n\'est pas au bon format. Il sera ignoré.'.format(height))
        return np.NaN
    else:
        value = height[:-1] # on enlève le dernier caractère, qui est 'm'
        return float(value)

# remplacement de valeur nulle par une valeur moyenne (attention, déforme les données )
def fill_height(height, replacement):
    if pd.isnull(height):
        print('Imputation par la moyenne : {}'.format(replacement))
        return replacement
    return height

In [4]:
# APPLICATION DU NETTOYAGE SUR DONNEES par actualisation de colonnes 
# data["nom_colonne"] = nouvelle_colonne
data['email'] = data['email'].apply(first)
data['pays'] = data['pays'].apply(check_country)

data['taille'] = [convert_height(t) for t in data['taille']]
data['taille'] = [t if t<3 else np.NaN for t in data['taille']] #suppression données aberrantes
mean_height = data['taille'].mean()
data['taille'] = [fill_height(t, mean_height) for t in data['taille']] #remplissage

data['date_naissance'] = pd.to_datetime(data['date_naissance'], 
                                           format='%d/%m/%Y', errors='coerce') #détruit infos si pas bon format
print(data)

 - Il y a plusieurs parties dans "['marco23@example.com', ' mc23@supermail.eu']", ne gardons que marco23@example.com.
 - "nan" n'est pas un pays valide, nous le supprimons.
 - "24" n'est pas un pays valide, nous le supprimons.
153cm n'est pas au bon format. Il sera ignoré.
Imputation par la moyenne : 1.52
Imputation par la moyenne : 1.52
   prenom                     email date_naissance           pays  taille
0   Leila         leila@example.com     1990-01-23         France    1.49
1  Samuel    samuel_329@example.com     2001-09-20            NaN    1.67
2   Radia  choupipoune@supermail.eu            NaT  Côte d'ivoire    1.52
3    Marc       marco23@example.com     1978-02-10         France    1.65
4    Heri   helloworld@supermail.eu     2008-03-05     Madagascar    1.34
5   Hanna    hanna2019@supermail.eu     1970-01-01            NaN    1.52
6  samuël    samuel_329@example.com            NaT          Bénin    1.45


In [ ]:
# NOTAS SUPPLEMENTAIRES

# REVISER LES COMPREHENSION DE LISTES !!

#  data['taille'] = [convert_height(t) for t in data['taille']]

# == 

# nouvelle_colonne = []
# for t in data['taille']:
#    nouvelle_colonne.append(convert_height(t))
# data['taille'] = nouvelle_colonne

# ==

# data['taille'] = data['taille'].apply(convert_height)